<a href="https://colab.research.google.com/github/shruthibandyadka/dl_classify_drosophila_ovarioles/blob/main/Cropping%2C_masking%2C_padding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install deepcell

     |████████████████████████████████| 140 kB 8.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 679 kB 45.1 MB/s 
     |████████████████████████████████| 123 kB 52.9 MB/s 
     |████████████████████████████████| 454.4 MB 9.1 kB/s 
     |████████████████████████████████| 37.1 MB 52 kB/s 
     |████████████████████████████████| 40 kB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 4.0 MB 53.1 MB/s 
     |████████████████████████████████| 1.2 MB 44.8 MB/s 
     |████████████████████████████████| 462 kB 70.8 MB/s 
  Created wheel for deepcell: filename=DeepCell-0.10.0-py3-none-any.whl size=234299 sha256=e1685f11a00bb6ef97aeec4258d17c687548f1af583c26214bf115c83afc3922
  Stored in directory: /root/.cache/pip/wheels/4a/5d/20/dafefcafccb5dfb71

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import skimage.io as io 
from skimage.measure import label,regionprops, regionprops_table


In [5]:
from deepcell.datasets.tracked import hela_s3_bench

(X_train, y_train), (X_test, y_test) = hela_s3_bench.load_tracked_data(seed=0)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


637083648/637081600 [==============================] - 49s 0us/step
X_train shape: (14, 40, 216, 256, 1)
X_test shape: (4, 40, 216, 256, 1)


In [6]:
x = X_train[1,8]
y= y_train[1,8]
y = y.astype('int')
roi = regionprops(y[...,0])


In [7]:
x = X_train[1,10]
y = y_train[1,10]

In [209]:
def crop (x,y):
    y = y.astype('int')
    bounding_box = regionprops(y[...,0])
    num = len(bounding_box)
    crop_x = []
    crop_x_mask = []
    crop_y = []
    for i in range(num):
        #fig,ax = plt.subplots(1,2)
        coords = bounding_box[i].bbox
        #print(coords)
        cropped_x = x[coords[0]:coords[2],coords[1]:coords[3]]
        crop_x.append(cropped_x)
        cropped_y = y[coords[0]:coords[2],coords[1]:coords[3]]
        crop_y.append(cropped_y)
        #ax[0].imshow(cropped_x[...,0])
        #print(cropped_x.shape, cropped_y.shape)
        #print(bounding_box[i].label)
        mask_im= (cropped_y != bounding_box[i].label)
        crop_x_mask.append(mask_im)
        #ax[1].imshow(mask_im[...,0])
        #print(mask_im.shape)
        cropped_x[mask_im] = 0
    return crop_x, crop_y, crop_x_mask

In [210]:
out_x, out_y, mask_x = crop(X_train[1,10], y_train[1,10])


In [211]:
a = len(out_x)
x_dim = []
y_dim = []
for i in range(a):
    listdim = out_x[i].shape
    x_dim.append(listdim[0])
    y_dim.append(listdim[1])
max_x = max(x_dim)
max_y = max(y_dim)

In [212]:
def padding (out_x):
    a = len(out_x)
    padded_x = []
    for i in range(a):
        shape_array = out_x[i].shape
        pad_x = shape_array[0]
        pad_y = shape_array[1]
        x_pad_dim = (max_x - pad_x)//2 
        rem_x = (max_x - pad_x)%2
        y_pad_dim = (max_y - pad_y)//2 
        rem_y = (max_y - pad_y)%2
        padded_array = np.pad(np.squeeze(out_x[i]), ((x_pad_dim, (x_pad_dim+rem_x)), (y_pad_dim, (y_pad_dim+rem_y))))
        padded_x.append(padded_array)
    return padded_x

In [204]:
padded_x_images = padding(out_x)

In [206]:
stacked_padded_masked_x_images = np.stack(padded_x_images)